In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pprint
import geopandas as gpd
import json
from shapely.geometry import shape, Point  
from geopandas import GeoDataFrame
from shapely.geometry import box
import random
import folium
from matplotlib.colors import to_hex
from keplergl import KeplerGl



In [10]:
info = pd.read_csv(
    "C:/AIDC/PythonProject/lec01/dataset/특수건물 자연재해 위험도 지수.csv", low_memory=False) #공공 데이터 CSV 파일은 utf-8이 아닌 cp949 또는 euc-kr 인코딩 필요

info.columns  = info.columns.str.lower()

info.columns = [
    "일련번호",              # SEQ_NO
    "적용년도",              # APLY_YEAR
    "침수취약도지수",        # F_BI_FLOOD
    "건물대수용물비",        # C
    "지하침수방어높이",      # i1_under
    "1층바닥높이",           # i2_ground
    "출입구방지턱높이",      # i2_doorSill
    "차수판높이",            # i5_blockWater
    "모래주머니비치",        # i5_sand_cd_nm
    "수장공사구성비",        # WaterConstructRate_MAX
    "수변전실위치",          # Elect_Loaction_Gather
    "배수펌프및집수정",      # i3_pump_cd_nm
    "건물골조",              # NRI_bdframe_NM
    "외벽마감재"             # NRI_Wall_NM
]

info

,일련번호,적용년도,침수취약도지수,건물대수용물비,지하침수방어높이,1층바닥높이,출입구방지턱높이,차수판높이,모래주머니비치,수장공사구성비,수변전실위치,배수펌프및집수정,건물골조,외벽마감재
0,1,2020,0.85,0,0.10,4.00,0.0,0.0,없음,3.13,지하층,1) 있음,철근콘크리트,콘크리트/모르타르
1,2,2020,0.93,0,0.15,0.05,0.0,0.0,없음,3.41,지하층,1) 있음,철근콘크리트,샌드위치판넬
2,3,2020,0.91,0,0.20,0.20,0.0,0.0,없음,2.62,지하층,1) 있음,철근콘크리트,조적조
3,4,2020,0.56,0,0.30,0.80,0.0,0.0,없음,3.46,지상층,1) 있음,철근콘크리트,조적조
4,5,2020,0.89,0,0.30,0.30,0.0,0.0,없음,3.46,지하층,1) 있음,철근콘크리트,석재타일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36896,36897,2020,0.60,0,0.00,2.00,0.0,0.0,없음,3.10,지상층,2) 없음,철골조,샌드위치판넬
36897,36898,2020,0.78,0,0.50,0.50,0.0,0.0,없음,5.58,지상층옥상,1) 있음,철근콘크리트,콘크리트/모르타르
36898,36899,2020,1.43,0,0.00,0.30,0.2,0.0,없음,5.60,옥상,2) 없음,철골조,샌드위치판넬
36899,36900,2020,0.94,0,0.00,0.80,0.0,0.0,없음,2.51,지상층,2) 없음,철골조,샌드위치판넬


In [13]:
# 필요한 컬럼만 추출
df_water = info[["지하침수방어높이","1층바닥높이", "침수취약도지수", "건물골조"]].copy()

# 결측치 제거
df_water = df_water.dropna()

# 확인
df_water.head()

df_water.to_csv("강남구_침수취약도지수.csv", index=False)

df_water

,지하침수방어높이,1층바닥높이,침수취약도지수,건물골조
0,0.10,4.00,0.85,철근콘크리트
1,0.15,0.05,0.93,철근콘크리트
2,0.20,0.20,0.91,철근콘크리트
3,0.30,0.80,0.56,철근콘크리트
4,0.30,0.30,0.89,철근콘크리트
...,...,...,...,...
36896,0.00,2.00,0.60,철골조
36897,0.50,0.50,0.78,철근콘크리트
36898,0.00,0.30,1.43,철골조
36899,0.00,0.80,0.94,철골조


In [4]:

# gangnam_geojson = {
#   "type": "FeatureCollection",
#   "generator": "overpass-turbo",
#   "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.",
#   "timestamp": "2025-06-19T07:27:32Z",
#   "features": [
#     {
#       "type": "Feature",
#       "properties": {
#         "@id": "relation/2410520",
#         "admin_level": "6",
#         "boundary": "administrative",
#         "name": "강남구",
#         "name:en": "Gangnam-gu",
#         "name:ja": "江南区",
#         "name:ko": "강남구",
#         "name:ko-Hani": "江南區",
#         "name:zh": "江南區",
#         "name:zh-Hans": "江南区",
#         "name:zh-Hant": "江南區",
#         "ref": "23",
#         "type": "boundary",
#         "wikidata": "Q20398",
#         "wikipedia": "ko:강남구"
#       },
#       "geometry": {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [
#               127.1160432,
#               37.4782623
#             ],
#             [
#               127.1148644,
#               37.4798408
#             ],
#             [
#               127.1124932,
#               37.4826335
#             ],
#             [
#               127.1121843,
#               37.4833894
#             ],
#             [
#               127.1116479,
#               37.4841343
#             ],
#             [
#               127.1108466,
#               37.4852471
#             ],
#             [
#               127.1084009,
#               37.4880194
#             ],
#             [
#               127.1060718,
#               37.4907454
#             ],
#             [
#               127.1039173,
#               37.492501
#             ],
#             [
#               127.1018674,
#               37.4935466
#             ],
#             [
#               127.1006564,
#               37.4941643
#             ],
#             [
#               127.0969297,
#               37.4958274
#             ],
#             [
#               127.0929701,
#               37.4972134
#             ],
#             [
#               127.0918597,
#               37.4975705
#             ],
#             [
#               127.0886612,
#               37.4985993
#             ],
#             [
#               127.0880502,
#               37.4987791
#             ],
#             [
#               127.0850509,
#               37.4996619
#             ],
#             [
#               127.0831635,
#               37.5002366
#             ],
#             [
#               127.0820476,
#               37.5003636
#             ],
#             [
#               127.0797789,
#               37.5011141
#             ],
#             [
#               127.0769737,
#               37.5020645
#             ],
#             [
#               127.0738018,
#               37.5023648
#             ],
#             [
#               127.0729016,
#               37.50245
#             ],
#             [
#               127.0714859,
#               37.502583
#             ],
#             [
#               127.0698183,
#               37.5027396
#             ],
#             [
#               127.0688686,
#               37.5103019
#             ],
#             [
#               127.067145,
#               37.5208743
#             ],
#             [
#               127.0668037,
#               37.5233479
#             ],
#             [
#               127.0665922,
#               37.524881
#             ],
#             [
#               127.0651947,
#               37.5254814
#             ],
#             [
#               127.0640966,
#               37.5260257
#             ],
#             [
#               127.0628655,
#               37.5266359
#             ],
#             [
#               127.057204,
#               37.5301151
#             ],
#             [
#               127.0556451,
#               37.5309305
#             ],
#             [
#               127.0520349,
#               37.5325467
#             ],
#             [
#               127.0492745,
#               37.5334867
#             ],
#             [
#               127.0461251,
#               37.5345591
#             ],
#             [
#               127.0405637,
#               37.5362407
#             ],
#             [
#               127.0328191,
#               37.5385032
#             ],
#             [
#               127.0292671,
#               37.5394266
#             ],
#             [
#               127.0259481,
#               37.5399807
#             ],
#             [
#               127.02391,
#               37.5397498
#             ],
#             [
#               127.021115,
#               37.5383185
#             ],
#             [
#               127.0175196,
#               37.5349759
#             ],
#             [
#               127.0121629,
#               37.5296492
#             ],
#             [
#               127.0083762,
#               37.5258834
#             ],
#             [
#               127.0131754,
#               37.5226498
#             ],
#             [
#               127.0151068,
#               37.5247296
#             ],
#             [
#               127.0173529,
#               37.5222075
#             ],
#             [
#               127.0177396,
#               37.5217732
#             ],
#             [
#               127.0180871,
#               37.5212326
#             ],
#             [
#               127.0186625,
#               37.519479
#             ],
#             [
#               127.0193099,
#               37.5170298
#             ],
#             [
#               127.0196065,
#               37.5160017
#             ],
#             [
#               127.0200957,
#               37.5144311
#             ],
#             [
#               127.0201834,
#               37.5138429
#             ],
#             [
#               127.0205303,
#               37.5126801
#             ],
#             [
#               127.0212566,
#               37.5110547
#             ],
#             [
#               127.0232468,
#               37.5070917
#             ],
#             [
#               127.0244262,
#               37.504383
#             ],
#             [
#               127.0275374,
#               37.4979285
#             ],
#             [
#               127.0314772,
#               37.4895514
#             ],
#             [
#               127.0315908,
#               37.4893399
#             ],
#             [
#               127.0325764,
#               37.48731
#             ],
#             [
#               127.032751,
#               37.4869539
#             ],
#             [
#               127.0327586,
#               37.4869383
#             ],
#             [
#               127.0330227,
#               37.4863998
#             ],
#             [
#               127.0331333,
#               37.4861741
#             ],
#             [
#               127.0332523,
#               37.4859313
#             ],
#             [
#               127.0333758,
#               37.4856794
#             ],
#             [
#               127.0335132,
#               37.4853992
#             ],
#             [
#               127.0339455,
#               37.4845174
#             ],
#             [
#               127.0339997,
#               37.4844069
#             ],
#             [
#               127.0414897,
#               37.485425
#             ],
#             [
#               127.0439526,
#               37.4796797
#             ],
#             [
#               127.0450736,
#               37.4771319
#             ],
#             [
#               127.0509668,
#               37.4714321
#             ],
#             [
#               127.0487837,
#               37.4698201
#             ],
#             [
#               127.0502768,
#               37.4698906
#             ],
#             [
#               127.0510741,
#               37.4693406
#             ],
#             [
#               127.0511585,
#               37.4685362
#             ],
#             [
#               127.051018,
#               37.4674723
#             ],
#             [
#               127.0508192,
#               37.466867
#             ],
#             [
#               127.0526434,
#               37.4677506
#             ],
#             [
#               127.055518,
#               37.4685881
#             ],
#             [
#               127.0573518,
#               37.4683728
#             ],
#             [
#               127.0595058,
#               37.4691588
#             ],
#             [
#               127.0612184,
#               37.4689286
#             ],
#             [
#               127.0626553,
#               37.4691168
#             ],
#             [
#               127.0636988,
#               37.4689653
#             ],
#             [
#               127.0652152,
#               37.4695591
#             ],
#             [
#               127.0670459,
#               37.469661
#             ],
#             [
#               127.0686227,
#               37.4707806
#             ],
#             [
#               127.0701035,
#               37.470975
#             ],
#             [
#               127.0718123,
#               37.4718737
#             ],
#             [
#               127.0727387,
#               37.4719554
#             ],
#             [
#               127.0738092,
#               37.472805
#             ],
#             [
#               127.0740981,
#               37.472906
#             ],
#             [
#               127.0752336,
#               37.4733682
#             ],
#             [
#               127.0763691,
#               37.4743155
#             ],
#             [
#               127.076864,
#               37.4750318
#             ],
#             [
#               127.0772425,
#               37.4750087
#             ],
#             [
#               127.0780141,
#               37.4747892
#             ],
#             [
#               127.0790068,
#               37.4748215
#             ],
#             [
#               127.0797711,
#               37.4749217
#             ],
#             [
#               127.0823041,
#               37.4752452
#             ],
#             [
#               127.0832358,
#               37.4754763
#             ],
#             [
#               127.0845168,
#               37.4753838
#             ],
#             [
#               127.0846624,
#               37.4731194
#             ],
#             [
#               127.0853029,
#               37.4713402
#             ],
#             [
#               127.0859435,
#               37.4703928
#             ],
#             [
#               127.0877486,
#               37.4698151
#             ],
#             [
#               127.0883891,
#               37.4683593
#             ],
#             [
#               127.0891461,
#               37.4680358
#             ],
#             [
#               127.0896119,
#               37.4673888
#             ],
#             [
#               127.091155,
#               37.4669959
#             ],
#             [
#               127.0916208,
#               37.4660947
#             ],
#             [
#               127.0920867,
#               37.4647544
#             ],
#             [
#               127.0917936,
#               37.4629524
#             ],
#             [
#               127.0925816,
#               37.461727
#             ],
#             [
#               127.0931057,
#               37.4611955
#             ],
#             [
#               127.0943285,
#               37.4609644
#             ],
#             [
#               127.0961336,
#               37.460895
#             ],
#             [
#               127.0958425,
#               37.4598551
#             ],
#             [
#               127.0959589,
#               37.4587458
#             ],
#             [
#               127.0964248,
#               37.4576826
#             ],
#             [
#               127.0964539,
#               37.4564808
#             ],
#             [
#               127.0983014,
#               37.4565506
#             ],
#             [
#               127.1003806,
#               37.4574965
#             ],
#             [
#               127.1004901,
#               37.4576066
#             ],
#             [
#               127.1016918,
#               37.4584242
#             ],
#             [
#               127.101945,
#               37.459327
#             ],
#             [
#               127.1027174,
#               37.459838
#             ],
#             [
#               127.1036616,
#               37.4599402
#             ],
#             [
#               127.1041336,
#               37.4605193
#             ],
#             [
#               127.1046413,
#               37.4614523
#             ],
#             [
#               127.1043643,
#               37.4620629
#             ],
#             [
#               127.1052065,
#               37.4621884
#             ],
#             [
#               127.1058932,
#               37.4623588
#             ],
#             [
#               127.1078905,
#               37.4620219
#             ],
#             [
#               127.1081891,
#               37.4619662
#             ],
#             [
#               127.1124421,
#               37.4612346
#             ],
#             [
#               127.1131459,
#               37.4608931
#             ],
#             [
#               127.1137467,
#               37.4607569
#             ],
#             [
#               127.1149376,
#               37.4592324
#             ],
#             [
#               127.116018,
#               37.4586278
#             ],
#             [
#               127.1173028,
#               37.4582462
#             ],
#             [
#               127.1196626,
#               37.4634488
#             ],
#             [
#               127.1209929,
#               37.4651094
#             ],
#             [
#               127.1226881,
#               37.4669573
#             ],
#             [
#               127.1227254,
#               37.4677331
#             ],
#             [
#               127.1224342,
#               37.4686112
#             ],
#             [
#               127.1215026,
#               37.4702288
#             ],
#             [
#               127.1198139,
#               37.4727705
#             ],
#             [
#               127.1176594,
#               37.4760978
#             ],
#             [
#               127.1164906,
#               37.4777795
#             ],
#             [
#               127.1160432,
#               37.4782623
#             ]
#           ]
#         ]
#       },
#       "id": "relation/2410520"
#     },
#     {
#       "type": "Feature",
#       "properties": {
#         "@id": "node/1912558042",
#         "@relations": [
#           {
#             "role": "label",
#             "rel": "2410520",
#             "reltags": {
#               "admin_level": "6",
#               "boundary": "administrative",
#               "name": "강남구",
#               "name:en": "Gangnam-gu",
#               "name:ja": "江南区",
#               "name:ko": "강남구",
#               "name:ko-Hani": "江南區",
#               "name:zh": "江南區",
#               "name:zh-Hans": "江南区",
#               "name:zh-Hant": "江南區",
#               "ref": "23",
#               "type": "boundary",
#               "wikidata": "Q20398",
#               "wikipedia": "ko:강남구"
#             }
#           }
#         ]
#       },
#       "geometry": {
#         "type": "Point",
#         "coordinates": [
#           127.0473,
#           37.5177
#         ]
#       },
#       "id": "node/1912558042"
#     }
#   ]
# }

# # Feature 하나만 있으므로 바로 사용
# geometry = [shape(gangnam_geojson["features"][0]["geometry"])]
# properties = [gangnam_geojson["features"][0]["properties"]]

# gdf = GeoDataFrame(properties, geometry=geometry)
# gdf.set_crs(epsg=4326, inplace=True)


# gdf.plot()


# 강남구와 겹치는(포함되는) 객체만 필터링
- 뭔가 확인 필요

In [5]:

# 1. 강남구 GeoJSON 불러오기
with open("C:/AIDC/PythonProject/lec01/dataset/export.geojson", encoding="utf-8") as f:
    gangnam_geojson = json.load(f)

# 2. GeoDataFrame으로 변환
gangnam_geometry = [shape(gangnam_geojson["features"][0]["geometry"])]
gangnam_gdf = gpd.GeoDataFrame(geometry=gangnam_geometry, crs="EPSG:4326")

# # 3. 침수 위험 SHP 데이터 불러오기
# inundation_gdf = gpd.read_file("C:/AIDC/PythonProject/lec01/dataset/CFM_SAREA_1_030.shp")

# # 4. 좌표계 통일 (강남구와 동일한 EPSG:4326로 변환)
# inundation_gdf = inundation_gdf.to_crs(gangnam_gdf.crs)

# # 5. 강남구와 겹치는(포함되는) 객체만 필터링
# inundation_in_gangnam = inundation_gdf[inundation_gdf.intersects(gangnam_gdf.iloc[0].geometry)]

# # 6. 결과 확인
# inundation_in_gangnam.plot()

# 위 코드 시각화

In [6]:
# from shapely.geometry import Point
# # 1. 좌표 추출
# coords = []
# for geom in inundation_in_gangnam.geometry:
#     if geom.geom_type == "Polygon":
#         coords.extend(list(geom.exterior.coords))
#     elif geom.geom_type == "MultiPolygon":
#         for poly in geom.geoms:
#             coords.extend(list(poly.exterior.coords))

# # 2. 좌표를 DataFrame으로 변환
# coord_df = pd.DataFrame(coords, columns=["lon", "lat"])

# # 3. 좌표를 포인트로 변환한 GeoDataFrame 생성
# coord_gdf = gpd.GeoDataFrame(geometry=[Point(xy) for xy in coords], crs=inundation_in_gangnam.crs)

# # 4. 시각화 및 결과 확인
# coord_gdf.plot()

# coord_df.to_csv("강남구_침수_경계좌표.csv", index=False, encoding="utf-8-sig")

# 위 변환으로 생긴 "강남구_침수_경계좌표.csv"

In [7]:
coord_df = pd.read_csv("강남구_침수_경계좌표.csv", encoding="utf-8-sig")
print(coord_df.shape)
coord_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '강남구_침수_경계좌표.csv'

In [ ]:
# 1km 당 위도/경도 변화량
LAT_GRID_SIZE = 0.009    # 위도 기준: 1km ≈ 0.009도
LON_GRID_SIZE = 0.0113   # 경도 기준: 1km ≈ 0.0113도

# 기준점 설정 (옵션: 첫 번째 좌표 또는 최소값)
origin_lat = coord_df['lat'].min()
origin_lon = coord_df['lon'].min()

# 격자 인덱스 계산
coord_df['grid_x'] = ((coord_df['lon'] - origin_lon) // LON_GRID_SIZE).astype(int)
coord_df['grid_y'] = ((coord_df['lat'] - origin_lat) // LAT_GRID_SIZE).astype(int)

# 그룹 ID 부여 (문자열 또는 정수 결합)
coord_df['group'] = coord_df['grid_x'].astype(str) + "_" + coord_df['grid_y'].astype(str)

# 결과 확인
print(coord_df.shape)
coord_df.head()

# 시각화를 위한 그룹 평균 중심 좌표 추출

In [ ]:
# 그룹별 위도/경도 평균 및 개수 계산
grouped_df = coord_df.groupby('group').agg({
    'lat': 'mean',
    'lon': 'mean',
    'group': 'count'   # 그룹 내 행 수
}).rename(columns={'group': 'count'}).reset_index()

# 컬럼 순서 정리
grouped_df = grouped_df[['group', 'lat', 'lon', 'count']]

# 결과 확인
print(grouped_df.shape)
print(f"💡 총 그룹 수: {len(grouped_df)}")
grouped_df.head()

## keplergl 시각화
- 대량 마커 효율적 (18.4 MB - 인스톨타임 3Min)

In [ ]:
# !pip install keplergl

In [ ]:
# -------------------------------------
# 2. 시각화 설정 (블루 계열, point + polygon)
# -------------------------------------
config = {
    "version": "v1",
    "config": {
        "visState": {
            "layers": [
                {
                    "id": "points_layer",
                    "type": "point",
                    "config": {
                        "dataId": "Group Centers + Count",
                        "label": "Group Centers + Count",
                        "color": [30, 100, 200],
                        "columns": {
                            "lat": "lat",
                            "lng": "lon",
                            "altitude": None
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 20,
                            "fixedRadius": False,
                            "opacity": 0.8,
                            "outline": False,
                            "thickness": 2,
                            "colorRange": {
                                "name": "ColorBrewer Blues-6",
                                "type": "sequential",
                                "category": "ColorBrewer",
                                "colors": ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#2171b5"]
                            },
                            "radiusRange": [1, 40]
                        },
                        "textLabel": []
                    },
                    "visualChannels": {
                        "colorField": {"name": "count", "type": "integer"},
                        "colorScale": "quantile",
                        "sizeField": {"name": "count", "type": "integer"},
                        "sizeScale": "linear"
                    }
                },
                {
                    "id": "gangnam_layer",
                    "type": "geojson",
                    "config": {
                        "dataId": "Gangnam GeoJSON",
                        "label": "Gangnam District",
                        "color": [70, 130, 180],
                        "highlightColor": [255, 255, 0, 255],
                        "columns": {
                            "geojson": "_geojson"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.2,
                            "thickness": 1,
                            "strokeColor": [70, 130, 180],
                            "colorRange": {
                                "name": "ColorBrewer Blues-6",
                                "type": "sequential",
                                "category": "ColorBrewer",
                                "colors": ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#2171b5"]
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]
                            },
                            "radius": 10
                        },
                        "textLabel": []
                    },
                    "visualChannels": {}
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "Group Centers + Count": ["group", "count", "lat", "lon"],
                        "Gangnam GeoJSON": []
                    },
                    "enabled": True
                }
            },
            "layerBlending": "normal",
            "splitMaps": []
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": grouped_df['lat'].mean(),
            "longitude": grouped_df['lon'].mean(),
            "pitch": 0,
            "zoom": 12,
            "isSplit": False
        },
        "mapStyle": {
            "styleType": "dark",
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": False,
                "water": True,
                "land": True
            }
        }
    }
}

# -------------------------------------
# 3. Kepler 지도 생성 및 데이터 추가
# -------------------------------------
map_1 = KeplerGl(height=600, config=config)

# 그룹 중심 포인트 추가
map_1.add_data(data=grouped_df, name="Group Centers + Count")

# 강남구 폴리곤 GeoJSON 추가
map_1.add_data(data=gangnam_gdf, name="Gangnam GeoJSON")
map_1.save_to_html(file_name='시각화_초기_전국구_kepler_map.html')
# 지도 표시
map_1.show()

In [ ]:
info = pd.read_csv(
    "C:/AIDC/PythonProject/lec01/dataset/침수유발_기준강우량_데이터_2023(서울).csv", low_memory=False) #공공 데이터 CSV 파일은 utf-8이 아닌 cp949 또는 euc-kr 인코딩 필요

info.columns  = info.columns.str.lower()

# 필요한 컬럼만 추출
df_water = info[["sgg_nm", "inclination", "intercept","depth_10", "depth_20", "depth_50" ]].copy()

# 결측치 제거
df_water = df_water.dropna()

# '강남구' 데이터만 필터링
df_gangnam = df_water[df_water['sgg_nm'] == '강남구']

# 결과 확인
df_gangnam.head()

df_gangnam.to_csv("강남구_침수데이터.csv", index=False)
